<a href="https://colab.research.google.com/github/ga0808/sql/blob/main/%EC%8B%A4%EC%8A%B58.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#실습 1

# -- 아래는 GYM_MEMBER 테이블에 대한 구조를 보여줍니다. 테이블을 수정하실 필요는 없습니다.
DESC GYM_MEMBER;

# -- 스쿼트 점수, 벤치 프레스 점수, 데드 리프트 점수의 합계에 대해 동일 점수는 같은 순위로 취급하여 순위를 매겨 조회하세요.
SELECT
    MEMBER_ID, SQUAT, BENCH_PRESS, DEADLIFT,
    #세 점수들에 대한 합계 구하기
    (SQUAT + BENCH_PRESS + DEADLIFT) AS WEIGHT_SUM,
    #구한 합계 점수에 대해서 순위 매기기
    RANK() OVER (ORDER BY WEIGHT_SUM DESC) AS 'RANK'
FROM
    GYM_MEMBER;

In [ ]:
# 실습 2

# -- 아래는 PRODUCT, SELL 테이블에 대한 구조를 보여줍니다. 테이블을 수정하실 필요는 없습니다.
DESC PRODUCT;
DESC SELL;

# -- 1. SELL 테이블과 PRODUCT 테이블을 상품 ID 기준으로 INNER JOIN 하는 쿼리를 작성하세요.
SELECT SELL_ID, SELLER_NAME, PRODUCT_NAME, QUANTITY
FROM SELL
JOIN PRODUCT
ON SELL.PRODUCT_ID = PRODUCT.PRODUCT_ID
ORDER BY SELL_ID;

#1번 문항 처럼 두 테이블을 조인해서,
#SUM 집계 함수를 이용하여, 판매자 이름, 상품 ID 별 판매 금액을 구하는 쿼리를 작성하세요.
SELECT
    SELL_ID,  SELLER_NAME, PRODUCT_NAME, QUANTITY,
    #판매자 이름, 상품 id 두 컬럼의 조합 별로 -> 판매금액(PRICE * QUANTITY)을 집계하기 -> 집계는 SUM()
    SUM(PRODUCT.PRICE * SELL.QUANTITY) OVER (PARTITION BY SELLER_NAME, SELL.PRODUCT_ID) AS 'SUM_PRICE'
FROM
    SELL
JOIN PRODUCT
ON SELL.PRODUCT_ID = PRODUCT.PRODUCT_ID
ORDER BY SELL_ID;

In [ ]:
# 실습 3

# -- 아래는 PARTICIPANT 테이블에 대한 구조를 보여줍니다. 테이블을 수정하실 필요는 없습니다.
DESC PARTICIPANT;

# -- 그룹 별, 참가자의 앞, 뒤 기록과 그룹 내에서 가장 빠른 기록, 가장 느린 기록을 조회하세요.
SELECT
    ID,
    GROUP_NUM, #그룹 번호
    TIME_RECORD, #시간 기록
    #그룹 내 참가자의 직전=앞의 기록 조회
    LAG(TIME_RECORD, 1) OVER( #1행 앞의, 시간 기록을 조회
        PARTITION BY GROUP_NUM  # 그룹별로
        ORDER BY TIME_RECORD) AS 'FRONT', #시간 기록, 오름 차순 정렬 = 빠른 기록 부터 정렬 (정렬해서, 1행 앞 조회)
    #그룹 내 참가자의 직후=뒤의 기록 조회
    LEAD(TIME_RECORD, 1) OVER( #1행 뒤의, 시간 기록을 조회
        PARTITION BY GROUP_NUM #그룹별로
        ORDER BY TIME_RECORD) AS 'BEHIND', #시간 기록, 오름 차순 정렬 = 빠른 기록 부터 정렬 (정렬해서, 1행 뒤 조회)
    #그룹 내 가장 빠른 기록
    FIRST_VALUE(TIME_RECORD) OVER( #맨 첫번째 행의 값=시간 기록 조회
        PARTITION BY GROUP_NUM #그룹별로
        ORDER BY TIME_RECORD  ##시간 기록, 오름 차순 정렬 = 빠른 기록 부터 정렬 -> 빠른 기록이 맨 앞 = 첫번째 행으로 존재
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS 'FIRST_VALUE', #맨 첫번째 행 ~ 맨 마지막 행 모두 참조해서
    #그룹 내 가장 느린 기록
    LAST_VALUE(TIME_RECORD) OVER( #맨 마지막 행의 값 = 시간 기록 조회
        PARTITION BY GROUP_NUM
        ORDER BY TIME_RECORD #빠른 기록 부터 정렬하니까, 맨 마지막 기록은 늦은 기록
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS 'LAST_VALUE'
FROM
    PARTICIPANT
ORDER BY
    GROUP_NUM, TIME_RECORD

In [ ]:
# 실습 7

# -- 아래는 BOOK_HISTORY 테이블에 대한 구조를 보여줍니다. 테이블을 수정하실 필요는 없습니다.
DESC BOOK_HISTORY;

# -- 책 종류와 책 카테고리로 그룹화 하여 각 그룹당 판매 개수 합을 구해보세요.
SELECT kind, category, sum(sell_count) # 집계
from BOOK_HISTORY
group by kind, category #책의 종류 - 카테고리 조합으로 그룹화
ORDER by sum(sell_count);

In [ ]:
# 실습 8

# -- 아래는 BOOK_HISTORY 테이블에 대한 구조를 보여줍니다. 테이블을 수정하실 필요는 없습니다.
DESC BOOK_HISTORY;

# -- 책 종류와 책 카테고리로 그룹화 하고 ROLLUP을 이용하여 각 그룹당 판매 개수 합을 구해보세요.
SELECT kind, category, sum(sell_count)
from BOOK_HISTORY
group by kind, category with ROLLUP;
#책 종류 마다 집계 수행 , 책-카테고리 조합별로 집계 수행 , 전체 데이터로 집계 수행

In [ ]:
# 미션 1

# -- 아래는 STUDENT 테이블에 대한 구조를 보여줍니다. 테이블을 수정하실 필요는 없습니다.
DESC STUDENT;

# -- 키와 이름에 대해 오름차순으로 학생들을 정렬시켜 봅시다.
SELECT
    #키와 이름으로 오름 차순 정렬하여, ROW_NUMBER 순위 매기기
    ROW_NUMBER() OVER(ORDER BY HEIGHT, NAME) AS ROW_NUMBER,
    NAME,
    HEIGHT
FROM
    STUDENT;


In [ ]:
# 미션 3
# -- 아래는 STUDENT, COURSE_SCORE 테이블에 대한 구조를 보여줍니다. 테이블을 수정하실 필요는 없습니다.
DESC STUDENT;
DESC COURSE_SCORE;

#1. join

SELECT *
FROM COURSE_SCORE CS
JOIN STUDENT S -- AS 생략하고, 테이블에 대한 별칭 설정
ON CS.STUDENT_ID = S.ID; -- 설정한 별칭으로 테이블 명명



# -- 수업이름과 학과에 대해서 ROLLUP 을 이용하여 평균 점수를 구해보세요.
SELECT
    CS.COURSE_NAME,
    S.DEPARTMENT,
    AVG(CS.SCORE) AS AVG
FROM COURSE_SCORE CS
JOIN STUDENT S
ON CS.STUDENT_ID = S.ID
GROUP BY CS.COURSE_NAME, S.DEPARTMENT WITH ROLLUP
#수업 이름 계층 집계, 수업 이름과 학과의 계층 집계, 모든 데이터 집계
;